In [214]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
from datetime import date, timedelta
import io
import time
import sqlite3
import pandas as pd
import csv

## 뉴스 크롤링

In [215]:
def make_date_list(year):
    # 시작 날짜와 종료 날짜 설정
    start_date = date(year, 1, 1)
    if year == 2023:
        end_date = date.today()
    else:
        end_date = date(year, 12, 31)

    # 날짜 리스트 초기화
    date_list = []

    # 시작 날짜부터 종료 날짜까지 반복하면서 날짜를 리스트에 추가
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime('%Y%m%d')
        date_list.append(date_str)
        current_date += timedelta(days=1)
        
    return date_list

In [216]:
press_list = ['032', '005', '020', '021', '081', '022', '023', '025', '028', '469']

In [217]:
make_date_list(2023)[0][-2:]

'01'

In [219]:
def news_crawling(year):
    headers = {'user-agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
    f = open('news.csv', 'w', newline='', encoding='utf-8')
    wtr = csv.writer(f)
    wtr.writerow(["date", "press", "title", "url"])
    
    date_list = make_date_list(year)

    for date in tqdm(date_list): # 날짜별 반복
        
        if date[-2:] == '15':
            time.sleep(60)
        else:
            time.sleep(1)
                
        for press in press_list: # 신문사별 반복
                
            url = f'https://media.naver.com/press/{press}/ranking?type=popular&date={date}'
            try:
                rs = requests.get(url, headers=headers)
            except:
                print('RETRY-TIME ERROR!')
                time.sleep(5)
                rs = requests.get(url, headers=headers)
            
            rs.encoding = 'UTF-8'
            soup = BeautifulSoup(rs.text, 'html.parser')
            
            link_list = []
            title_list = []

            titles = soup.find_all("strong", "list_title")
            for title in titles:
                title_list.append(title.get_text())
            
            links = soup.find_all("li", {'class': ['as_thumb', 'as_no_thumb']})
            for link in links:
                link_list.append(link.find('a')['href'])
            
            for i in range(20):
                try:
                    wtr.writerow([date, press, title_list[i], link_list[i]])
                except:
                    print(date, press)

In [220]:
news_crawling(2023)

 75%|████████████████████████████████████████████████████████████▎                   | 241/320 [26:35<06:13,  4.73s/it]

20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022
20230830 022


100%|████████████████████████████████████████████████████████████████████████████████| 320/320 [35:51<00:00,  6.72s/it]


In [221]:
df = pd.read_csv("news.csv")

In [222]:
df

,date,press,title,url
0,20230101,32,“아내와 함께 떠난 딸…마지막이 될 줄 몰랐다”,https://n.news.naver.com/article/032/000319644...
1,20230101,32,"뜸 들이다 끝난 ‘더 글로리’···복수한다며, 남자가 왜 거기서 나와[리뷰]",https://n.news.naver.com/article/032/000319637...
2,20230101,32,이것은 발 토시가 아니다…세련미 더해 돌아온 ‘라떼는’ 패션,https://n.news.naver.com/article/032/000319630...
3,20230101,32,"경찰, ‘방음터널 화재’ 폐기물 운반용 집게 트럭 소유 업체 압수수색",https://n.news.naver.com/article/032/000319636...
4,20230101,32,다시 만난 ‘자낳세’…“월급만으로 살 수 없단 생각엔 변함 없어”,https://n.news.naver.com/article/032/000319644...
...,...,...,...,...
63975,20231116,469,"""한동훈 탄핵 필요하면 검토""... '검사 탄핵' 물러서지 않는 민주당",https://n.news.naver.com/article/469/000077058...
63976,20231116,469,제2의 손흥민·이강인도 '전국대회 8강' 못 들면 대학 못 간다?,https://n.news.naver.com/article/469/000077038...
63977,20231116,469,尹대통령에 '쪽지메모' 전달한 정대철 “검사출신 넘어 폭넓은 인사 등용하라”[정치행간],https://n.news.naver.com/article/469/000077045...
63978,20231116,469,"[르포]""MMORPG 한계 왔다""...택진이형까지 새 장르·플랫폼으로 위기 탈출 나섰다",https://n.news.naver.com/article/469/000077061...


In [224]:
df['url'][0]

'https://n.news.naver.com/article/032/0003196443?ntype=RANKING'

In [253]:
3200 % (len(df['url']) // 20)

1

In [284]:
def news_content_crawling(data):
    headers = {'user-agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
    f = open('news_content.csv', 'w', newline='', encoding='utf-8')
    wtr = csv.writer(f)
    wtr.writerow(["title", "content"])
    
    for i in range(len(data["url"])):
        if i % (len(data['url']) // 20) == 100:
            time.sleep(60)
        else:
            time.sleep(1)
        
        url = data['url'][i]
        try:
            rs = requests.get(url, headers=headers)
        except:
            print('RETRY-TIME ERROR!')
            time.sleep(5)
            rs = requests.get(url, headers=headers)
            
        if rs.status_code != 200:
            print('CONTENT REQUEST FAILED! : ', url)
            return "REQUEST_ERROR"
        
        rs.encoding = 'UTF-8' # html의 인코딩 방식 확인하기
        soup = BeautifulSoup(rs.text, 'html.parser')
        
        title = soup.select_one('#title_area > span').get_text()
        print(title)
        raw_txt = soup.select_one('#newsct_article')
        if raw_txt != None:
            for i in raw_txt.find_all('span'):
                i.decompose()
            raw_txt = raw_txt.get_text(separator='\n')
            #e-mail/기자 제거
            pt = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
            raw_txt = re.sub(pattern=pt, repl='', string=raw_txt)
            raw_txt = raw_txt.replace('기자', '')
            raw_txt = raw_txt.strip()
        
            wtr.writerow([title, raw_txt])
        else:
            
            return "CONTENT_CRAWL_ERROR"

In [285]:
news_content_crawling(df)

“아내와 함께 떠난 딸…마지막이 될 줄 몰랐다”
뜸 들이다 끝난 ‘더 글로리’···복수한다며, 남자가 왜 거기서 나와[리뷰]
CONTENT REQUEST FAILED! :  https://n.news.naver.com/article/032/0003196308?ntype=RANKING


'REQUEST_ERROR'

In [ ]:
def content_crawl(u):
    headers = {'user-agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
    try:
        rs = requests.get(u, headers=headers)
    except:
        print('RETRY-TIME ERROR!')
        time.sleep(3)
        rs = requests.get(u, headers=headers)
    if rs.status_code != 200:
        print('CONTENT REQUEST FAILED! : ', u)
        return "REQUEST_ERROR"
    rs.encoding = 'UTF-8' # html의 인코딩 방식 확인하기
    soup = BeautifulSoup(rs.text, 'html.parser')
    raw_txt = soup.select_one('#newsct_article')
    if raw_txt != None:
        for i in raw_txt.find_all('span'):
            i.decompose()
        raw_txt = raw_txt.text
        #e-mail/기자 제거
        pt = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
        raw_txt = re.sub(pattern=pt, repl='', string=raw_txt)
        raw_txt = raw_txt.replace('기자', '')
        raw_txt = raw_txt.strip()
        return raw_txt
    else:
        return "CONTENT_CRAWL_ERROR"